# Quantization of embeddings
## Motivation
## Mathematical background

- have values in interval [min, max]
- transform to integers in range [a, b]

$x\in [min, max]$
$\tilde{x} = a + \frac{b - a}{max - min}(x - min) = a + \frac{1}{\Delta}(x - min)$ mit $\Delta = \frac{max - min}{b - a}$
- apply round, ceil or floor: $\overline{x} = round(\tilde{x})$

In [ ]:
from collections.abc import Callable
import math

min_v = -1
max_v = 1
a = -128
b = 127

delta = (max_v - min_v) / (b - a)

d = 5

def forward_transformation(x: float, transform: Callable[[float], int] = math.floor) -> int:
    x_tilde = a + (b - a) / (max_v - min_v) * (x - min_v)
    return transform(x_tilde)

backward transformation
$\hat{x} = (\overline{x} - a) * \Delta + min$

In [13]:
def backward_transformation(x: int) -> float:
    return (x - a) * (max_v - min_v) / (b - a) + min_v

## Estimation of the quantization error

- $\vert x - \hat{x}\vert$
- $\vert \langle x, y\rangle - \langle\hat{x}, \hat{y}\rangle\vert$

In [14]:
import random
import math
import numpy as np


def pnorm(x: list[float], p: int = 2) -> float:
    return math.pow(sum(np.abs(x_i) ** p for x_i in x), 1 / p)

def normalize(x: list[float]) -> list[float]:
    norm = pnorm(x, 2)
    return [x_i / norm for x_i in x]

x = [random.random() for _ in range(d)]
y = [random.random() for _ in range(d)]

x = normalize(x)
y = normalize(y)

x_bar = [forward_transformation(x_i) for x_i in x]
y_bar = [forward_transformation(y_i) for y_i in y]

x_hat = [backward_transformation(x_i) for x_i in x_bar]
y_hat = [backward_transformation(y_i) for y_i in y_bar]

In [15]:
integer_dot_product = sum(x_i * y_i for (x_i, y_i) in zip(x_bar, y_bar))

dot_product_x_part = min_v * delta * (sum(x_i for x_i in x_bar) - d * a) - a * delta ** 2 * sum(x_i for x_i in x_bar)
dot_product_y_part = min_v * delta * (sum(y_i for y_i in y_bar) - d * a) - a * delta ** 2 * sum(y_i for y_i in y_bar)

In [16]:
dot_product = np.dot(x, y)
dot_product_hat = np.dot(x_hat, y_hat)
dot_product_hat_improved = delta ** 2 * integer_dot_product + dot_product_x_part + dot_product_y_part + d * a ** 2 * delta ** 2 + d * min_v ** 2

print("<x,y> =", dot_product)
print("<x_hat,y_hat> = ", dot_product_hat)
print("Improved calculation: ", dot_product_hat_improved)

print("Difference = ", np.abs(dot_product - dot_product_hat))
print("Error estimation = ", delta * (pnorm(x_hat, 1) + pnorm(y_hat, 1)) + d * delta ** 2)
print("Error estimation 2 = ", 2 * delta * math.sqrt(d) + d * delta ** 2)

<x,y> = 0.6603766820913078
<x_hat,y_hat> =  0.6397078046905038
Improved calculation:  0.6397078046905023
Difference =  0.02066887740080403
Error estimation =  0.030265282583621683
Error estimation 2 =  0.035383150127639915


## Efficient calculation of the dot product